In [9]:
# NB17 — Repo root autodetect (data/ + artifacts/ locator)
from pathlib import Path

def find_repo_root(start: Path, must_have=("data", "artifacts")) -> Path:
    cur = start.resolve()
    for _ in range(6):
        if all((cur / m).exists() for m in must_have):
            return cur
        cur = cur.parent
    if start.name.lower() == "notebooks" and all((start.parent / m).exists() for m in must_have):
        return start.parent.resolve()
    raise FileNotFoundError(f"Could not locate repo root containing {must_have} starting at {start}")

CWD = Path.cwd()
ROOT = find_repo_root(CWD)
print("Repo root →", ROOT)
print("Has data?      ", (ROOT/"data").exists())
print("Has artifacts? ", (ROOT/"artifacts").exists())


Repo root → C:\.projects\stock-direction-ml\notebooks
Has data?       True
Has artifacts?  True


In [10]:
# NB17 — Write requirements.txt pinned to current versions
import importlib, sys

pkgs = [
    "streamlit",
    "scikit-learn",
    "pandas",
    "numpy",
    "joblib",
    "matplotlib",
    "pyarrow",       # for parquet read
    # "fastparquet", # uncomment if you use it
]

lines = []
for name in pkgs:
    try:
        mod = importlib.import_module(name if name != "scikit-learn" else "sklearn")
        ver = getattr(mod, "__version__", None)
        if ver is None and name == "joblib":
            ver = importlib.import_module("joblib").__version__
        if ver:
            lines.append(f"{name}=={ver}")
        else:
            lines.append(name)  # fallback unpinned
    except Exception:
        lines.append(name)      # not installed right now; leave unpinned

req_path = (ROOT / "requirements.txt")
req_path.write_text("\n".join(lines) + "\n", encoding="utf-8")
print("Wrote:", req_path.resolve())
print(req_path.read_text(encoding="utf-8"))


Wrote: C:\.projects\stock-direction-ml\notebooks\requirements.txt
streamlit
scikit-learn==1.7.2
pandas==2.3.2
numpy==2.3.3
joblib==1.5.2
matplotlib==3.10.6
pyarrow==21.0.0



In [11]:
# NB17 — Write .streamlit/config.toml (basic settings/theme)
from pathlib import Path

cfg_dir = (ROOT / ".streamlit")
cfg_dir.mkdir(parents=True, exist_ok=True)
cfg_path = cfg_dir / "config.toml"

cfg = """\
[server]
headless = true
runOnSave = false

[theme]
base = "light"
primaryColor = "#0E7AFE"
backgroundColor = "#FFFFFF"
secondaryBackgroundColor = "#F6F8FB"
textColor = "#0B132B"
"""

cfg_path.write_text(cfg, encoding="utf-8")
print("Wrote:", cfg_path.resolve())
print(cfg_path.read_text(encoding="utf-8"))


Wrote: C:\.projects\stock-direction-ml\notebooks\.streamlit\config.toml
[server]
headless = true
runOnSave = false

[theme]
base = "light"
primaryColor = "#0E7AFE"
backgroundColor = "#FFFFFF"
secondaryBackgroundColor = "#F6F8FB"
textColor = "#0B132B"



In [ ]:
# NB17 — Write README_STREAMLIT.md with deploy instructions (closed with ''' at the end)
from pathlib import Path

# Detect whether app lives at <ROOT>/app or notebooks/app
app_root_first = (ROOT / "app" / "streamlit_app.py")
app_notebooks  = (ROOT / "notebooks" / "app" / "streamlit_app.py")
if app_root_first.exists():
    app_path_cloud = "app/streamlit_app.py"
elif app_notebooks.exists():
    app_path_cloud = "notebooks/app/streamlit_app.py"
else:
    app_path_cloud = "app/streamlit_app.py"  # default

readme = f'''# Streamlit App — Deploy Guide

## Run locally
```bash
streamlit run {app_path_cloud}
'''